# Individual assignment: Analysis of the Impact of covid in Nicaragua, Mexico and Peru (January - September 2020) - the Latin American regions where Microwd operates

The impact of covid in Nicaragua, Mexico and Peru is going to be performed as follows:

1. PySpark **environment setup**
2. Data source and **Spark data abstraction** (DataFrame) **set up**
3. Data set **metadata analysis**:
  1. Display **schema and size** of the DataFrame
  2. Get one or multiple **random samples** from the data set to better understand what the data is all about
  3. Identify **data entities**, **metrics** and **dimensions**
  4. **Columns/fields categorization**
4. Columns groups **basic profiling** to better understand our data set:
  1. **Timing related** columns basic profiling
  2. **Geography related** columns basic profiling
  3. **Issue related** columns basic profiling
5. **Answer some business questions** to improve service
  1. **Top severly hit regions per country** (and figures) from January onwards
  2. **Ratio of top hit regions** and the rest of the country (on average) for all relevant countries
  3. **Months and weeks with the highest frequency of confirmed cases and deaths**


Let's go for it:

## 1. PySpark environment setup

In [1]:
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

ModuleNotFoundError: No module named 'findspark'

## 2. Data source and Spark data abstraction (DataFrame) setup

In [71]:
microwdcovidDF = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .csv("microwdcovid2_new.csv") # In order to perform a more detailed temporal analysis of the covid trends by month and week, 
# some new variables were created based on the date variable in the original dataset. # Furthermore, the original dataset 
# contained data for all the countries in the world, but I narrowed the scope to just the few Latin American countries
# where Microwd operates.

## 3. Data set metadata analysis
### A. Display schema and size of the DataFrame

In [72]:
from IPython.display import display, Markdown

microwdcovidDF.printSchema()
display(Markdown("This DataFrame has **%d rows**." % microwdcovidDF.count()))

root
 |-- SNo: integer (nullable = true)
 |-- ObservationDate: string (nullable = true)
 |-- Province_State: string (nullable = true)
 |-- Country_Region: string (nullable = true)
 |-- LastUpdate: string (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- date_iso: string (nullable = true)
 |-- date_month: integer (nullable = true)
 |-- date_week: integer (nullable = true)



This DataFrame has **6746 rows**.

### B. Get one or multiple random samples from the data set

In [73]:
microwdcovidDF.cache() # optimization to make the processing faster
microwdcovidDF.sample(False, 0.1).take(2)

[Row(SNo=5487, ObservationDate='03/14/2020', Province_State=None, Country_Region='Peru', LastUpdate='2020-03-14T12:33:03', Confirmed=38, Deaths=0, Recovered=0, date_iso='14/03/2020', date_month=3, date_week=11),
 Row(SNo=5740, ObservationDate='03/15/2020', Province_State=None, Country_Region='Peru', LastUpdate='2020-03-15T18:20:18', Confirmed=43, Deaths=0, Recovered=0, date_iso='15/03/2020', date_month=3, date_week=11)]

### C. Data entities, metrics and dimensions

I've identified the following elements:

* **Entities:** *Confirmed, *Deaths*, *Recovered*
* **Metrics:** *ObservationDate*, *date_iso*, *date_month*, *date_week*...
* **Dimensions:** *Country_Region*, *Province_State*

### D. Column categorization

The following could be a potential column categorization:

* **Timing related columns:** *ObservationDate*, *date_iso*, *date_month*, *date_week*...
* **Geographic location related columns:** *Country_Region*, *Province_State*
* **Issue related columns:** *Confirmed, *Deaths*, *Recovered*

## 4. Columns groups basic profiling to better understand our data set
### A. Basic profiling of columns related to time and issue

In [74]:
from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first, lit

In [75]:
print ("Summary of columns ObservationDate, Confirmed, Deaths, Recovered:")
microwdcovidDF.select("ObservationDate", "Confirmed", "Deaths", "Recovered").summary().show()

print("Checking for nulls on columns Date_Month, Date_Week, Province/State, Country/Region, Confirmed, Deaths, Recovered")
microwdcovidDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["date_month", "date_week", "Province_State", "Country_Region", "Confirmed", "Deaths", "Recovered"]]).show()

print("Checking amount of distinct values in columns Checking for nulls on columns Date_Month, Date_Week, Province/State, Country/Region, Confirmed, Deaths, Recovered:")
microwdcovidDF.select([countDistinct(c).alias(c) for c in ["date_month", "date_week", "Province_State", "Country_Region", "Confirmed", "Deaths", "Recovered"]]).show()

Summary of columns ObservationDate, Confirmed, Deaths, Recovered:
+-------+---------------+-----------------+------------------+-----------------+
|summary|ObservationDate|        Confirmed|            Deaths|        Recovered|
+-------+---------------+-----------------+------------------+-----------------+
|  count|           6746|             6746|              6746|             6746|
|   mean|           null|12358.34983694041| 908.2365846427513| 8772.99303290839|
| stddev|           null|28417.23397247107|1633.2652367246103|36850.66995595356|
|    min|     01/23/2020|                0|                 0|                0|
|    25%|           null|             2104|               106|                0|
|    50%|           null|             5382|               366|             1087|
|    75%|           null|           328903|             13504|           536959|
|    max|     09/10/2020|           328903|             13504|           536959|
+-------+---------------+-----------------+

## 5. Answer some business questions to improve service

In [76]:
# Checking the frequency by counting the number of rows per month and week

print ("Most and least frequent occurrences for ObservationDate column:")
Date_Month = microwdcovidDF.groupBy("date_month").agg(count(lit(1)).alias("Total"))
Date_Week = microwdcovidDF.groupBy("date_week").agg(count(lit(1)).alias("Total"))

leastFreqMonth    = Date_Month.orderBy(col("Total").asc()).first()
mostFreqMonth     = Date_Month.orderBy(col("Total").desc()).first()
leastFreqWeek     = Date_Week.orderBy(col("Total").asc()).first()
mostFreqWeek      = Date_Week.orderBy(col("Total").desc()).first()

display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("leastFreqMonth", "mostFreqMonth", "leastFreqWeek", "mostFreqWeek", \
       "%d (%d occurrences)" % (leastFreqMonth["date_month"], leastFreqMonth["Total"]), \
       "%d (%d occurrences)" % (mostFreqMonth["date_month"], mostFreqMonth["Total"]), \
       "%d (%d occurrences)" % (leastFreqWeek["date_week"], leastFreqWeek["Total"]), \
       "%d (%d occurrences)" % (mostFreqWeek["date_week"], mostFreqWeek["Total"]))))

Most and least frequent occurrences for ObservationDate column:



| leastFreqMonth | mostFreqMonth | leastFreqWeek | mostFreqWeek |
|----|----|----|----|
| 1 (1 occurrences) | 8 (1829 occurrences) | 4 (1 occurrences) | 34 (413 occurrences) |


In [77]:
## To check how many rows are per country

print ("Most and least frequent occurrences for ObservationDate column:")
Province_State = microwdcovidDF.groupBy("Province_State").agg(count(lit(1)).alias("Total"))
Country_Region = microwdcovidDF.groupBy("Country_Region").agg(count(lit(1)).alias("Total"))

leastFreqProvince_State    = Province_State.orderBy(col("Total").asc()).first()
mostFreqProvince_State     = Province_State.orderBy(col("Total").desc()).first()
leastFreqCountry_Region     = Country_Region.orderBy(col("Total").asc()).first()
mostFreqCountry_Region      = Country_Region.orderBy(col("Total").desc()).first()

display(Markdown("""
| %s | %s | %s | %s |
|----|----|----|----|
| %s | %s | %s | %s |
""" % ("leastFreqProvince_State ", "mostFreqProvince_State", "leastFreqCountry_Region", "mostFreqCountry_Region", \
       "%s (%d occurrences)" % (leastFreqProvince_State["Province_State"], leastFreqProvince_State["Total"]), \
       "%s (%d occurrences)" % (mostFreqProvince_State["Province_State"], mostFreqProvince_State ["Total"]), \
       "%s (%d occurrences)" % (leastFreqCountry_Region["Country_Region"], leastFreqCountry_Region["Total"]), \
       "%s (%d occurrences)" % (mostFreqCountry_Region["Country_Region"], mostFreqCountry_Region["Total"]))))

Most and least frequent occurrences for ObservationDate column:



| leastFreqProvince_State  | mostFreqProvince_State | leastFreqCountry_Region | mostFreqCountry_Region |
|----|----|----|----|
| Pasco (106 occurrences) | None (342 occurrences) | Nicaragua (176 occurrences) | Mexico (3731 occurrences) |


Total number of confirmed cases, deaths and recovered patients per country:

In [78]:
from pyspark.sql.functions import max, min, avg, stddev, sum

microwdcovidDF\
    .select("Country_Region", "Confirmed", "Deaths", "Recovered")\
    .groupBy("Country_Region")\
    .agg(sum("Confirmed").alias("Confirmed"), sum("Deaths").alias("Deaths"),\
        sum("Recovered").alias("Recovered"))\
    .select("Country_Region", "Confirmed", "Deaths", "Recovered").show()

+--------------+---------+-------+---------+
|Country_Region|Confirmed| Deaths|Recovered|
+--------------+---------+-------+---------+
|     Nicaragua|   328133|  10745|   214987|
|          Peru| 44045174|1781960| 28293450|
|        Mexico| 38996121|4334259| 30674174|
+--------------+---------+-------+---------+



### A. Top severely hit regions
   #### A.1 Top severly hit regions overall

In [79]:
#Severity is computed with regards to the total number of confirmed cases and deaths. 

microwdcovidDF\
    .where(col("Province_State").isNotNull())\
    .select("Province_State", "Confirmed", "Deaths")\
    .groupBy("Province_State")\
    .agg(sum("Confirmed").alias("Confirmed"), sum("Deaths").alias("Deaths"))\
    .orderBy(col("Confirmed").desc())\
    .select("Province_State", "Confirmed", "Deaths").show()

+----------------+---------+------+
|  Province_State|Confirmed|Deaths|
+----------------+---------+------+
|            Lima| 21410137|772197|
|Ciudad de Mexico|  6973099|827157|
|          Mexico|  4856447|563915|
|          Callao|  2154571| 97222|
|           Piura|  2074635|117939|
|         Tabasco|  1843789|174606|
|        Veracruz|  1806835|241427|
|      Guanajuato|  1723171| 94838|
|          Puebla|  1716279|217310|
|      Lambayeque|  1685899|111369|
|        Arequipa|  1546332| 61307|
|      Nuevo Leon|  1513368|105064|
|     La Libertad|  1447516|114960|
|          Sonora|  1396179|152570|
|      Tamaulipas|  1338299| 93847|
| Baja California|  1267755|248119|
|             Ica|  1207856| 84140|
|         Jalisco|  1184463|136620|
|          Ancash|  1175843| 74818|
|         Sinaloa|  1159995|193710|
+----------------+---------+------+
only showing top 20 rows



 #### A.2 Top severly hit regions per country
    Peru:

In [80]:
microwdcovidDF\
    .where(col("Province_State").isNotNull())\
    .where(col("Country_Region") == "Peru")\
    .select("Country_Region", "Province_State", "Confirmed", "Deaths")\
    .groupBy("Country_Region", "Province_State")\
    .agg(sum("Confirmed").alias("Confirmed"), sum("Deaths").alias("Deaths"))\
    .orderBy(col("Confirmed").desc())\
    .select("Country_Region", "Province_State", "Confirmed", "Deaths").show()

+--------------+--------------+---------+------+
|Country_Region|Province_State|Confirmed|Deaths|
+--------------+--------------+---------+------+
|          Peru|          Lima| 21410137|772197|
|          Peru|        Callao|  2154571| 97222|
|          Peru|         Piura|  2074635|117939|
|          Peru|    Lambayeque|  1685899|111369|
|          Peru|      Arequipa|  1546332| 61307|
|          Peru|   La Libertad|  1447516|114960|
|          Peru|           Ica|  1207856| 84140|
|          Peru|        Ancash|  1175843| 74818|
|          Peru|        Loreto|  1126506| 55891|
|          Peru|       Ucayali|   987033| 21296|
|          Peru|    San Martin|   790425| 32150|
|          Peru|         Junin|   768191| 33529|
|          Peru|     Cajamarca|   650925| 18659|
|          Peru|       Huanuco|   599115| 16624|
|          Peru|         Cusco|   570685| 11216|
|          Peru|      Amazonas|   522782| 11127|
|          Peru|      Ayacucho|   390909|  9659|
|          Peru|    

    Mexico:

In [81]:
microwdcovidDF\
    .where(col("Province_State").isNotNull())\
    .where(col("Country_Region") == "Mexico")\
    .select("Country_Region", "Province_State", "Confirmed", "Deaths")\
    .groupBy("Country_Region", "Province_State")\
    .agg(sum("Confirmed").alias("Confirmed"), sum("Deaths").alias("Deaths"))\
    .orderBy(col("Confirmed").desc())\
    .select("Country_Region", "Province_State", "Confirmed", "Deaths").show()

+--------------+----------------+---------+------+
|Country_Region|  Province_State|Confirmed|Deaths|
+--------------+----------------+---------+------+
|        Mexico|Ciudad de Mexico|  6973099|827157|
|        Mexico|          Mexico|  4856447|563915|
|        Mexico|         Tabasco|  1843789|174606|
|        Mexico|        Veracruz|  1806835|241427|
|        Mexico|      Guanajuato|  1723171| 94838|
|        Mexico|          Puebla|  1716279|217310|
|        Mexico|      Nuevo Leon|  1513368|105064|
|        Mexico|          Sonora|  1396179|152570|
|        Mexico|      Tamaulipas|  1338299| 93847|
|        Mexico| Baja California|  1267755|248119|
|        Mexico|         Jalisco|  1184463|136620|
|        Mexico|         Sinaloa|  1159995|193710|
|        Mexico|        Coahuila|  1120183| 65755|
|        Mexico|        Guerrero|   942270|119786|
|        Mexico|       Michoacan|   916235| 71959|
|        Mexico|          Oaxaca|   888948| 83328|
|        Mexico| San Luis Potos

    Nicaragua:

In [82]:
## The same cannot be done for Nicaragua because there are no registered regions for Peru in the COVID dataset. 
# See report for further information. 

microwdcovidDF\
    .where(col("Province_State").isNotNull())\
    .where(col("Country_Region") == "Nicaragua")\
    .select("Country_Region", "Province_State", "Confirmed", "Deaths")\
    .groupBy("Country_Region", "Province_State")\
    .agg(sum("Confirmed").alias("Confirmed"), sum("Deaths").alias("Deaths"))\
    .orderBy(col("Confirmed").desc())\
    .select("Country_Region", "Province_State", "Confirmed", "Deaths").show()

+--------------+--------------+---------+------+
|Country_Region|Province_State|Confirmed|Deaths|
+--------------+--------------+---------+------+
+--------------+--------------+---------+------+



### B. Weeks with the highest rate of confirmed cases and deaths per country.

    Mexico:

In [83]:
microwdcovidDF\
.where(col("Country_Region") == "Mexico")\
    .select("Country_Region", "date_week", "Confirmed", "Deaths")\
    .groupBy("Country_Region", "date_week")\
    .agg(sum("Confirmed").alias("Confirmed"), sum("Deaths").alias("Deaths"))\
    .orderBy(col("Confirmed").desc())\
    .select("Country_Region", "date_week", "Confirmed", "Deaths").show()

+--------------+---------+---------+------+
|Country_Region|date_week|Confirmed|Deaths|
+--------------+---------+---------+------+
|        Mexico|       36|  4394742|470145|
|        Mexico|       35|  4132689|445448|
|        Mexico|       34|  3879277|420181|
|        Mexico|       33|  3612348|393964|
|        Mexico|       32|  3322735|362762|
|        Mexico|       31|  3016278|330148|
|        Mexico|       30|  2685009|301695|
|        Mexico|       29|  2361762|270747|
|        Mexico|       28|  2057399|242067|
|        Mexico|       27|  1754303|211680|
|        Mexico|       26|  1484590|183082|
|        Mexico|       37|  1299685|138698|
|        Mexico|       25|  1228012|147788|
|        Mexico|       24|   997008|117652|
|        Mexico|       23|   792069| 93356|
|        Mexico|       22|   612987| 67569|
|        Mexico|       21|   458829| 49929|
|        Mexico|       20|   330155| 34684|
|        Mexico|       19|   231905| 23142|
|        Mexico|       18|   154

    Peru:

In [84]:
microwdcovidDF\
.where(col("Country_Region") == "Peru")\
    .select("Country_Region", "date_week", "Confirmed", "Deaths")\
    .groupBy("Country_Region", "date_week")\
    .agg(sum("Confirmed").alias("Confirmed"), sum("Deaths").alias("Deaths"))\
    .orderBy(col("Confirmed").desc())\
    .select("Country_Region", "date_week", "Confirmed", "Deaths").show()

+--------------+---------+---------+------+
|Country_Region|date_week|Confirmed|Deaths|
+--------------+---------+---------+------+
|          Peru|       36|  4726505|206596|
|          Peru|       35|  4411356|199212|
|          Peru|       34|  4030867|190700|
|          Peru|       33|  3632944|173975|
|          Peru|       32|  3281620|145799|
|          Peru|       31|  2918190|135106|
|          Peru|       30|  2650251|121980|
|          Peru|       29|  2443641| 90782|
|          Peru|       28|  2262031| 81782|
|          Peru|       27|  2092859| 72856|
|          Peru|       26|  1928641| 63919|
|          Peru|       25|  1754154| 54727|
|          Peru|       24|  1559032| 44991|
|          Peru|       37|  1398966| 60359|
|          Peru|       23|  1341217| 37162|
|          Peru|       22|  1087814| 30459|
|          Peru|       21|   813930| 23662|
|          Peru|       20|   616635| 17702|
|          Peru|       19|   448393| 12595|
|          Peru|       18|   298

    Nicaragua:

In [85]:
microwdcovidDF\
.where(col("Country_Region") == "Nicaragua")\
    .select("Country_Region", "date_week", "Confirmed", "Deaths")\
    .groupBy("Country_Region", "date_week")\
    .agg(sum("Confirmed").alias("Confirmed"), sum("Deaths").alias("Deaths"))\
    .orderBy(col("Confirmed").desc())\
    .select("Country_Region", "date_week", "Confirmed", "Deaths").show()

+--------------+---------+---------+------+
|Country_Region|date_week|Confirmed|Deaths|
+--------------+---------+---------+------+
|     Nicaragua|       36|    32826|   990|
|     Nicaragua|       35|    31632|   963|
|     Nicaragua|       34|    30360|   935|
|     Nicaragua|       33|    29001|   901|
|     Nicaragua|       32|    27527|   866|
|     Nicaragua|       31|    25934|   819|
|     Nicaragua|       30|    24306|   764|
|     Nicaragua|       29|    22321|   702|
|     Nicaragua|       28|    20223|   645|
|     Nicaragua|       27|    17960|   589|
|     Nicaragua|       26|    15539|   527|
|     Nicaragua|       25|    13108|   458|
|     Nicaragua|       24|    10607|   394|
|     Nicaragua|       37|     9636|   288|
|     Nicaragua|       23|     8172|   331|
|     Nicaragua|       22|     5672|   256|
|     Nicaragua|       21|     2408|   137|
|     Nicaragua|       20|      404|    65|
|     Nicaragua|       19|      121|    38|
|     Nicaragua|       18|      

### C .Ratio of average confirmed cases and deaths between most severely hit regions and countries?

In [86]:
from pyspark.sql.functions import max, min, avg, stddev, round, mean
from pyspark.sql.types import IntegerType

# Glimpse of average of deaths and confirmed cases per country:

averages = microwdcovidDF\
    .select("Country_Region", "Confirmed", "Deaths")\
    .groupBy("Country_Region")\
    .agg(sum("Confirmed").alias("Confirmed"), sum("Deaths").alias("Deaths"))\
    .orderBy(col("Country_Region").desc())\
    .select("Country_Region", "Confirmed", "Deaths").show()

+--------------+---------+-------+
|Country_Region|Confirmed| Deaths|
+--------------+---------+-------+
|          Peru| 44045174|1781960|
|     Nicaragua|   328133|  10745|
|        Mexico| 38996121|4334259|
+--------------+---------+-------+



Average per country:

In [87]:
average_Mexico = microwdcovidDF\
    .where(col("Country_Region") == "Mexico")\
    .select("Country_Region", "Confirmed", "Deaths")\
    .groupBy("Country_Region")\
    .agg(avg("Confirmed").alias("Confirmed"), avg("Deaths").alias("Deaths"))\
    .orderBy(col("Country_Region").desc())\
    .select("Country_Region", round("Confirmed", 2).alias("Confirmed_Mexico"), round("Deaths", 2).alias("Deaths_Mexico"))

average_Peru = microwdcovidDF\
    .where(col("Country_Region") == "Peru")\
    .select("Country_Region", "Confirmed", "Deaths")\
    .groupBy("Country_Region")\
    .agg(avg("Confirmed").alias("Confirmed"), avg("Deaths").alias("Deaths"))\
    .orderBy(col("Country_Region").desc())\
    .select("Country_Region", round("Confirmed", 2).alias("Confirmed_Peru"), round("Deaths", 2).alias("Deaths_Peru"))

average_Nicaragua = microwdcovidDF\
    .where(col("Country_Region") == "Nicaragua")\
    .select("Country_Region", "Confirmed", "Deaths")\
    .groupBy("Country_Region")\
    .agg(avg("Confirmed").alias("Confirmed"), avg("Deaths").alias("Deaths"))\
    .orderBy(col("Country_Region").desc())\
    .select("Country_Region", round("Confirmed", 2).alias("Confirmed_Nicaragua"), round("Deaths", 2).alias("Deaths_Nicaragua"))

Ratio comparison for Peru:  

In [88]:
from pyspark.sql.functions import max, min, avg, stddev, round, mean

In [89]:
province_Peru = microwdcovidDF\
    .where((col("Province_State") == "Arequipa") | (col("Province_State") == "San Martin") | (col("Province_State") == "Tacna") | (col("Province_State") == "Puno"))\
    .select("Country_Region", "Province_State", "Confirmed", "Deaths")\
    .groupBy("Country_Region", "Province_State")\
    .agg(mean("Confirmed").alias("Confirmed"), mean("Deaths").alias("Deaths"))\
    .select("Country_Region", "Province_State", round("Confirmed", 2).alias("Confirmed"), round("Deaths", 2).alias("Deaths"))

In [90]:
Peru_combineDf = province_Peru.join(average_Peru, province_Peru["Country_Region"] == average_Peru["Country_Region"])

Peru_combineDf\
    .select("Province_State", "Confirmed", "Deaths", "Confirmed_Peru", "Deaths_Peru", (col("Confirmed")/col("Confirmed_Peru")).alias("RatioConfirmedPeru"), (col("Deaths")/col("Deaths_Peru")).alias("RatioDeathsPeru")).show()

+--------------+---------+------+--------------+-----------+-------------------+-------------------+
|Province_State|Confirmed|Deaths|Confirmed_Peru|Deaths_Peru| RatioConfirmedPeru|    RatioDeathsPeru|
+--------------+---------+------+--------------+-----------+-------------------+-------------------+
|          Puno|  3004.98| 84.11|      15514.33|     627.67|0.19369060732883728| 0.1340035368904042|
|         Tacna|  3164.29| 47.36|      15514.33|     627.67| 0.2039591783854024|0.07545366195612345|
|      Arequipa| 14588.04|578.37|      15514.33|     627.67| 0.9402945534870021| 0.9214555419248969|
|    San Martin|  7456.84| 303.3|      15514.33|     627.67|  0.480642090248177|0.48321570251883955|
+--------------+---------+------+--------------+-----------+-------------------+-------------------+



Ratio comparison for Mexico:

In [91]:
from pyspark.sql.functions import max, min, avg, stddev, round, mean

province_Mexico = microwdcovidDF\
    .where((col("Province_State") == "Campeche") | (col("Province_State") == "Yucatan"))\
    .select("Country_Region", "Province_State", "Confirmed", "Deaths")\
    .groupBy("Country_Region", "Province_State")\
    .agg(mean("Confirmed").alias("Confirmed"), mean("Deaths").alias("Deaths"))\
    .select("Country_Region", "Province_State", "Confirmed", "Deaths")

In [92]:
Mexico_combineDf = province_Mexico.join(average_Mexico, province_Mexico["Country_Region"] == average_Mexico["Country_Region"])

Mexico_combineDf\
    .select("Province_State", "Confirmed", "Deaths", "Confirmed_Mexico", "Deaths_Mexico", (col("Confirmed")/col("Confirmed_Mexico")).alias("RatioConfMex"), (col("Deaths")/col("Deaths_Mexico")).alias("RatioDeathsMex"))\
    .select("Province_State", round("Confirmed", 2).alias("Confirmed"),\
    round("Deaths", 2).alias("Deaths"),\
    round("RatioConfMex", 2).alias("RatioConfMex"), round("RatioDeathsMex", 2).alias("RatioDeathsMex")).show()

+--------------+---------+------+------------+--------------+
|Province_State|Confirmed|Deaths|RatioConfMex|RatioDeathsMex|
+--------------+---------+------+------------+--------------+
|       Yucatan|  7553.85|691.91|        0.72|           0.6|
|      Campeche|  3135.63|384.44|         0.3|          0.33|
+--------------+---------+------+------------+--------------+

